# Import Packages

In [108]:
from strava_api import StravaAPI
from garmin_file_manager import CleanData,WrangleData, GarminDataManager
import pandas as pd
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Strava Data

In [109]:
# init the api with the access token
strava = StravaAPI()

strava_data_pulls = []
# through trial and error I know I have about 8 pages of data. We could probably automate this but for now this is fine
page_nums = 8
for page_num in range(page_nums):
    strava_data_pulls.append(pd.json_normalize(strava.get_dataset(page_num = page_num + 1)))
    # I don't want to overload my API limits, so I'll give it some time in between requests
    time.sleep(.5)
strava_data = pd.concat(strava_data_pulls, ignore_index = True)

Requesting Token...



In [113]:
strava_data.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'average_temp', 'has_heartrate',
       'average_heartrate', 'max_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'athlete.id',
       'athlete.resource_state', 'map.id', 'map.summary_polyline',
       'map.resource_state', 'average_watts', 'kilojoules', 'device_watts'],


## Load Garmin Data
There are two ways to get your activities from Garmin. The first is signing into your Garmin connect account, clicking your activities on the sidebar, scrolling down for all the activities that you want to select, then clicking export data in the top right corner. The other involves plugging your watch into your computer, copying and pasting the FIT files, then converting them to excel files with Garming SDK. The first option is easiest in my opinion

In [64]:
# read in the data from the csv files in data
# we might need to read every singele file in data then concat them?
file_path = 'data/' 
garmin_data = pd.read_csv(file_path + 'Activities_cole_munoz.csv')
# clean the data
garmin_data_clean = CleanData(garmin_data).data
# wrangle the data
garmin_data_wrangle = WrangleData(garmin_data_clean).data

In [114]:
garmin_data_wrangle.columns

Index(['Activity Type', 'Date', 'Favorite', 'Title', 'Distance', 'Calories',
       'Time', 'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence',
       'Max Run Cadence', 'Avg Pace', 'Best Pace', 'Total Ascent',
       'Total Descent', 'Avg Stride Length', 'Avg Vertical Ratio',
       'Avg Vertical Oscillation', 'Avg Ground Contact Time',
       'Avg Run Cadence.1', 'Max Run Cadence.1', 'Training Stress Score®',
       'Avg Power', 'Max Power', 'Grit', 'Flow', 'Avg. Swolf',
       'Avg Stroke Rate', 'Total Reps', 'Dive Time', 'Min Temp',
       'Surface Interval', 'Decompression', 'Best Lap Time', 'Number of Laps',
       'Max Temp', 'Moving Time', 'Elapsed Time', 'Min Elevation',
       'Max Elevation', 'Day', 'Hour', 'Minute', 'Weekday'],
      dtype='object')